In [475]:
import pandas as pd
import numpy as np
import random
courses= pd.ExcelFile("courses.xlsx")


In [476]:
courses.sheet_names[0]

'Robotics'

In [477]:
coursesDic=dict()
for sheetnum,sheet in enumerate(courses.sheet_names):
    
    # Parse data from each worksheet as a Pandas DataFrame and append it 
    # to the dic
    df = courses.parse(sheet)
    df.set_index('name',inplace=True)
    df["name_col"]=df.index
    coursesDic[courses.sheet_names[sheetnum]]=df

In [478]:
coursesDic2=coursesDic.copy()
coursesDic2["Fluid"].head()


,name_col
name,
ابرام عاطف جرجس كامل,ابرام عاطف جرجس كامل
احمد سعيد عبد الفتاح خليل مرسي,احمد سعيد عبد الفتاح خليل مرسي
احمد شعبان محمد احمد الفقي,احمد شعبان محمد احمد الفقي
احمد محمد محمد كامل,احمد محمد محمد كامل
احمد محمد مفتاح,احمد محمد مفتاح


In [479]:
students=pd.DataFrame(columns=['name'])
students.set_index('name',inplace=True)
students["e"]=0

for course in coursesDic.values():

  students=pd.merge(students,course, how='outer',left_index=True, right_index=True).dropna(axis=1)
  

students

""
name
ابرام عاطف جرجس كامل
ابرام عاطف لطفي وهبه
ابراهيم محمد عطفي محمد حسن
ابراهيم نصر شعبان نصر
احمد اسامه محمدعبد العزيز محمد
...
نور الدين طه محمود عبد العزيز
وئام عزالدين فؤاد فريد السيد
وليد محمود عبد الوهاب سيد


In [480]:
def isInCourse(course):

  try:
    return coursesDic[course].loc[students.index[116]].count()>0
  except:
    return False

isInCourse("Robotics")
studentMCourses=[]
for course in coursesDic2.keys():
  if isInCourse(course):
    studentMCourses.append(course)


In [481]:
def MAD(x):
  '''
  calculates the median absolute deviation which is defined as median(|x_i - median(x)|
  inputs: arraylike 
          X
  outputs: numpy float
          MAD
  '''
  return np.median(np.absolute(x - np.median(x)))


In [482]:

'''
def isIntersected(course1,course2):
  
 # checks the intersection of students between two courses
 # inputs: str
 #         course 1 name , course 2 name
 # outputs: bool
 #          True if there is intersection and False otherwise
  
  df_merge = pd.merge(coursesDic[course1], coursesDic[course2], how='inner',left_index=True, right_index=True)
  return not len(df_merge)==0

'''



"\ndef isIntersected(course1,course2):\n  \n # checks the intersection of students between two courses\n # inputs: str\n #         course 1 name , course 2 name\n # outputs: bool\n #          True if there is intersection and False otherwise\n  \n  df_merge = pd.merge(coursesDic[course1], coursesDic[course2], how='inner',left_index=True, right_index=True)\n  return not len(df_merge)==0\n\n"

In [483]:
def isTaken(course):
  '''
  checks whether the courses is already in the time table or not
  inputs: str
          course name
  outputs: bool
           True if exists in the table, False otherwise
  '''
  t1=any(timeTable["course1"].astype(str).str.match(course))
  t2=any(timeTable["course2"].astype(str).str.match(course))
  t3=any(timeTable["course3"].astype(str).str.match(course))

  return t1 or t2 or t3

In [484]:
'''timeTablelist = {'course1':[], 'course2': []}
timeTable = pd.DataFrame(data=timeTablelist)
for i in range(10000):
  randCourseI=random.choice(list(coursesDic.keys()))
  randCourseJ=random.choice(list(coursesDic.keys()))

  if(not isTaken(randCourseI) and  not isTaken(randCourseJ)):
    if(isIntersected(randCourseI,randCourseJ)==False):
      

      dayNCourses = pd.DataFrame([[randCourseI,randCourseJ]],
                                 columns=timeTable.columns)
      timeTable=timeTable.append(dayNCourses,ignore_index=True)
      coursesDic.pop(randCourseI)
      coursesDic.pop(randCourseJ)
'''

"timeTablelist = {'course1':[], 'course2': []}\ntimeTable = pd.DataFrame(data=timeTablelist)\nfor i in range(10000):\n  randCourseI=random.choice(list(coursesDic.keys()))\n  randCourseJ=random.choice(list(coursesDic.keys()))\n\n  if(not isTaken(randCourseI) and  not isTaken(randCourseJ)):\n    if(isIntersected(randCourseI,randCourseJ)==False):\n      \n\n      dayNCourses = pd.DataFrame([[randCourseI,randCourseJ]],\n                                 columns=timeTable.columns)\n      timeTable=timeTable.append(dayNCourses,ignore_index=True)\n      coursesDic.pop(randCourseI)\n      coursesDic.pop(randCourseJ)\n"

In [485]:


def isIntersected(*args):
  '''
  checks the intersection of students between two courses
  inputs: str
          course 1 name , course 2 name
  outputs: bool
           True if there is intersection and False otherwise
  '''

  df_merge=pd.concat(list(args))
  isduplicated=df_merge.duplicated()
  return not len(df_merge[isduplicated])==0


timeTableDic = {'course1':[], 'course2': [],'course3':[]}
timeTable = pd.DataFrame(data=timeTableDic)

for i in range(10000):
  try:
    randCourseI=random.choice(list(coursesDic.keys()))
    randCourseJ=random.choice(list(coursesDic.keys()))
    randCourseK=random.choice(list(coursesDic.keys()))
  except:
    None

  if(not isTaken(randCourseI) and  not isTaken(randCourseJ)and  not isTaken(randCourseK)):
    if(isIntersected(coursesDic2[randCourseI],coursesDic2[randCourseJ],coursesDic2[randCourseK])==False):
      

      dayNCourses = pd.DataFrame([[randCourseI,randCourseJ,randCourseK]],
                                 columns=timeTable.columns)
      timeTable=timeTable.append(dayNCourses,ignore_index=True)
      coursesDic.pop(randCourseI)
      coursesDic.pop(randCourseJ)
      coursesDic.pop(randCourseK)



for i in range(10000):
  try:
    randCourseI=random.choice(list(coursesDic.keys()))
    randCourseJ=random.choice(list(coursesDic.keys()))
  except:
    None

  if(not isTaken(randCourseI) and  not isTaken(randCourseJ)):
    if(isIntersected(coursesDic2[randCourseI],coursesDic2[randCourseJ])==False):
      

      dayNCourses = pd.DataFrame([[randCourseI,randCourseJ,"-"]],
                                 columns=timeTable.columns)
      timeTable=timeTable.append(dayNCourses,ignore_index=True)
      coursesDic.pop(randCourseI)
      coursesDic.pop(randCourseJ)


for i in range(10000):
  try:
    randCourseI=random.choice(list(coursesDic.keys()))
  except:
    None
    

  if(not isTaken(randCourseI)):


    coursesDic.pop(randCourseI)

    dayNCourses = pd.DataFrame([[randCourseI,"-","-"]],
                                columns=timeTable.columns)
    timeTable=timeTable.append(dayNCourses,ignore_index=True)


In [486]:
timeTable

,course1,course2,course3
0,Electronics1,InudstrialElectronics,MechanicsAndThermodynamics
1,Control1,SensorsAndActuators,Mechanics1
2,FundOfManuf,LinearAlgebra,Mechatronics1
3,accounting,Robotics,Thermodynamics
4,EngineeringDrawing,Microcontroller,Calc3
5,calc1,PLC,Logic
6,Ethics,AVS,Physics
7,CSAndProgramming,MachineDesign2,-
8,Signal,NumericalControl,-
9,FundOfMangement,ManufSystemAutomation,-


In [487]:
coursesDic.keys()

dict_keys([])

In [488]:
#shuffling
timeTable=timeTable.sample(frac = 1,ignore_index=True)
timeTable

,course1,course2,course3
0,accounting,Robotics,Thermodynamics
1,Fields,-,-
2,calc1,PLC,Logic
3,Control1,SensorsAndActuators,Mechanics1
4,FundOfManuf,LinearAlgebra,Mechatronics1
5,Vibration,-,-
6,Fluid,-,-
7,FundOfMangement,ManufSystemAutomation,-
8,CSAndProgramming,MachineDesign2,-
9,Signal,NumericalControl,-


In [489]:
studentMTT=timeTable[timeTable["course1"].astype(str).str.contains("|".join(studentMCourses), regex=True) | timeTable["course2"].astype(str).str.contains("|".join(studentMCourses), regex=True) ]
index=studentMTT.index
display(studentMTT)


,course1,course2,course3
0,accounting,Robotics,Thermodynamics
2,calc1,PLC,Logic
7,FundOfMangement,ManufSystemAutomation,-
9,Signal,NumericalControl,-
12,DigitalControl,-,-
13,Ethics,AVS,Physics


In [490]:
print(f"index MAD: {MAD(index)}")


index MAD: 4.5


In [491]:
print(f"left out courses are:\n{list(coursesDic.keys())}")

left out courses are:
[]


In [492]:
MAD([0,5,1,0,1]*12)

1.0

In [493]:
MAD([1,3,4,30])

1.5

In [494]:
MAD([3,6,9,12])

3.0